In [ ]:
"""
泰坦尼克号乘客生还预测（Titanic - Machine Learning from Disaster）是Kaggle机器学习的新手任务，本文将以它为例来完整介绍机器学习的一般流程
"""

**This is my fist journey in Machine Learning!**

By trandition, I will solve the classic "Titanic survival" problem by folowing four steps:

1. Import modules and datasets needed
2. Do an overview of datasets and a subsequent datasets preprocessing
3. Build models and select the better performer
4. Use the "best model" to do prediction 




*Submission File Format:*

*You should submit a csv file with exactly 418 entries plus a header row. Your submission will show an error if you have extra columns 
(beyond PassengerId and Survived) or rows.*

*The file should have exactly 2 columns:*

*1. PassengerId (sorted in any order)*

*2.Survived (contains your binary predictions: 1 for survived, 0 for deceased)*

In [ ]:
# Import modules and datasets needed 

# Basical API
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats import randint as sp_randint # give the integer random distribution 

# Sklearn support
from sklearn import preprocessing # dataset preprocess
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif # feature selection
from sklearn.model_selection import StratifiedKFold # k fold cross-validation
from sklearn.model_selection import cross_val_score 
from sklearn.linear_model import LogisticRegression # LR model
from sklearn.svm import SVC # SVC model
from sklearn.neighbors import KNeighborsClassifier # KNC model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis # LDA model
from sklearn.naive_bayes import GaussianNB # GNB model
from sklearn.tree import DecisionTreeClassifier # DTC model
from sklearn.ensemble import RandomForestClassifier # RFC model
from sklearn.model_selection import RandomizedSearchCV

# Import "Titanic-surviviors" datasets
train_df = pd.read_csv('../input/titanic/train.csv')
pred_df = pd.read_csv('../input/titanic/test.csv')

### tips for new kagglers ###

1. Before loading and reading the datasets, you must finish your phone verification, or you will get the error:"No such file in directory".

2. The "file (train.csv, test.csv, etc.) path" can be copied .

In [ ]:
# Do an overview of datasets and a subsequent datasets preprocessing 
train_df.info()
train_df.head()

pred_df.info()
pred_df.head()

# Drop feature
train_df = train_df.drop(labels = ["PassengerId","Name","Ticket","Cabin","Embarked"], axis = 1)
pred_df = pred_df.drop(labels = ["PassengerId","Name","Ticket","Cabin","Embarked"], axis = 1)

# Recode feature
lab_encoder = preprocessing.LabelEncoder().fit(["male","female"]) # "female -> 0 & male -> 1"
train_df["Sex"] = lab_encoder.transform(train_df["Sex"])
pred_df["Sex"] = lab_encoder.transform(pred_df["Sex"])

# Fullfill feature
combine_df = pd.concat([train_df.iloc[:,1:], pred_df.iloc[:,:]], axis = 0)
imputer = KNNImputer(n_neighbors=3).fit(combine_df)
train_df.iloc[:,1:] = imputer.transform(train_df.iloc[:,1:])
pred_df.iloc[:,:] = imputer.transform(pred_df.iloc[:,:])

# alter the type of features
train_df["Survived"] = train_df["Survived"].astype("int")
train_df["Pclass"] = train_df["Pclass"].astype("int")
train_df["Sex"] = train_df["Sex"].astype("object")
train_df["Parch"] = train_df["Parch"].astype("object")

pred_df["Pclass"] = pred_df["Pclass"].astype("int")
pred_df["Sex"] = pred_df["Sex"].astype("object")
pred_df["Parch"] = pred_df["Parch"].astype("object")

### Overview: ###

1. The features: "Name","Sex","Ticket","Cabin","Embarked" is non-numerical.

2. In train_df, the feature "Cabin" has a serious missing value problem (only 204 non-null), and "Age", "Embarked" have a slighter problem (714 and 899 non-null).

3. In pred_df, the feature "Cabin" has a serious missing value problem (only 91 non-null), and "Age", "Fare" have a slighter problem (332 and 417 non-null).

 ### Preprocess: ###
 
 1. Drop the features: “PassengerId","Name","Ticket","Cabin","Embarked",  because "Name","Ticket","Embarked" are almostly irrelatively with "survive or not", and the "cabin" has serious missing value problem in both two datasets.
 
 2. Fullfill missing values of "Age" and the noly missing value of "Fare" in pred_df by linear regression method.
 
 3. Recode the "Sex" (male, female) with number 0 and 1.

In [ ]:
# Build models and select the better performer 
Model = [LogisticRegression(), SVC(), KNeighborsClassifier(), LinearDiscriminantAnalysis(), GaussianNB(), DecisionTreeClassifier(),  RandomForestClassifier()]
Res_acc = []; Res_f1 = []
X_df = train_df.iloc[:,1:] # dataset of features (Pclass,Sex,...)
Y_df = train_df.iloc[:,0] # dataset of label (Survived)

for model in Model:
    kfold = StratifiedKFold(n_splits = 10, random_state = 1, shuffle=True)
    acc_results = cross_val_score(model, X_df, Y_df, cv = kfold, scoring = 'accuracy')
    Res_acc.append(round(acc_results.mean(),2))
    f1_results = cross_val_score(model, X_df, Y_df, cv = kfold, scoring = 'f1')
    Res_f1.append(round(f1_results.mean(),2))

print("accuracy for models:"+"\n"+  
"LogisticRegression: "+str(Res_acc[0])+"\n"+
"SVC: "+str(Res_acc[1])+"\n"+
"KNeighborsClassifier: "+str(Res_acc[2])+"\n"+
"LinearDiscriminantAnalysis: "+str(Res_acc[3])+"\n"+
"GaussianNB: "+str(Res_acc[4])+"\n"+
"DecisionTreeClassifier: "+str(Res_acc[5])+"\n"+
"RandomForestClassifier: "+str(Res_acc[6])+"\n"
)

print("f1-score for models:"+"\n"+  
"LogisticRegression: "+str(Res_f1[0])+"\n"+
"SVC: "+str(Res_f1[1])+"\n"+
"KNeighborsClassifier: "+str(Res_f1[2])+"\n"+
"LinearDiscriminantAnalysis: "+str(Res_f1[3])+"\n"+
"GaussianNB: "+str(Res_f1[4])+"\n"+
"DecisionTreeClassifier: "+str(Res_f1[5])+"\n"+
"RandomForestClassifier: "+str(Res_f1[6])
)

In [ ]:
# Select the best parameters of RandomForestClassifirer
param_dist1 = {"n_estimators":sp_randint(1,51),
              "max_depth": [3,4,5, None],                    
              "max_features": sp_randint(0, 11),          
              "min_samples_split": sp_randint(2, 11),    
              "bootstrap": [True, False],                 
              "criterion": ["gini", "entropy"]}

random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist1, n_iter=50, cv=10) # n_iter表示随机搜索20组，cv表示5折交叉验证
random_search.fit(X_df, Y_df)
print('best parameters:',random_search.best_params_,'\n','best score:', random_search.best_score_)

In [ ]:
# Use the "best model" to do prediction
RFC = RandomForestClassifier(
                        bootstrap = True,
                        criterion = 'gini',
                        n_estimators = 20,
                        max_depth = None,
                        max_features = 2,
                        min_samples_split = 10
).fit(X_df, Y_df)
pred_y = RFC.predict(pred_df)

print(pred_y)

df = pd.read_csv('../input/titanic/test.csv')
submission = pd.DataFrame({
        "PassengerId": df["PassengerId"],
        "Survived": pred_y
    })
submission.to_csv('./submission.csv', index=False)